In [3]:
import torch
import gensim

import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from os import listdir
from nltk import word_tokenize,pos_tag
from tqdm import tqdm_notebook as tqdm
from torch.autograd import Variable

In [4]:
dataset_path = 'dataset'
embedding_dimension = 100
embeddings_path = 'GoogleNews-vectors-negative300.bin'

In [6]:

dataset_files = listdir(dataset_path)

data_tokenized = []

for file_path in dataset_files:
    with open(dataset_path+'/'+file_path) as file:
        data_tokenized.append(word_tokenize(file.read()))

for i in tqdm(range(len(data_tokenized))):
        token_set = data_tokenized[i]
        datum_pos_tagged = pos_tag(token_set)
        for j in range(len(datum_pos_tagged)):
            tag = datum_pos_tagged[j][1]
            if(tag == 'NNP' or tag == 'NNPS'):
                data_tokenized[i][j] = '-pro-'

vocab = set([])
for data in data_tokenized:
    for token in data:
        vocab.add(token)

word2idx = {}
idx2word = {}

idx = 0
for word in vocab:
    word2idx[word] = idx
    idx2word[idx] = word
    idx += 1
    
idx -= 1
vocab_size = idx
initial_embeds = torch.rand(vocab_size,embedding_dimension)

# context, target
train_examples = []
window_size = 2

for i in tqdm(range(len(data_tokenized))):
    for j in range(len(data_tokenized[i])):
        for k in range(j-window_size,j+window_size+1):
            if(k<0 or j==k or k>=len(data_tokenized[i])):
                continue
            train_examples.append((word2idx[data_tokenized[i][k]],word2idx[data_tokenized[i][j]]))

In [ ]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_path, binary=True)  

In [18]:
# print(model.wv.vocab['run'])
print(model['run'].shape[0])
print(model.vocab['run'])
# word2idx[word] = idx
# idx2word[idx] = word

300
Vocab(count:2999791, index:209)


In [94]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embed1 = nn.Embedding(vocab_size,embedding_dimension)
        self.embed1.weight.requires_grad = True
        self.linear1 = nn.Linear(embedding_dimension,vocab_size,bias=False)
        self.linear1.weight.requires_grad = True
        
    def forward(self, x):
        x = self.embed1(x)
        x = self.linear1(x)
        return x

net = Net()

for name, param in net.named_parameters():
#     if param.requires_grad:
    print (name, param.data)

embed1.weight tensor([[-1.2616, -0.2915,  2.1652,  ...,  1.6965,  0.4838, -1.5713],
        [ 0.6393, -0.1153,  2.4186,  ...,  0.4038,  0.0099, -1.0737],
        [-0.7282, -0.6344, -0.1435,  ..., -1.7812, -0.4675, -0.1092],
        ...,
        [-0.2661, -0.1388,  1.4205,  ...,  0.1541,  0.3668, -0.1108],
        [-1.3831, -0.7770,  0.0425,  ..., -1.6575, -0.1327, -0.4992],
        [ 0.7098,  0.4363, -0.6173,  ..., -1.0384, -0.2999,  0.0213]])
linear1.weight tensor([[ 0.0302, -0.0172,  0.0684,  ..., -0.0449,  0.0554, -0.0743],
        [-0.0765,  0.0462, -0.0768,  ..., -0.0096, -0.0213,  0.0757],
        [-0.0556, -0.0989, -0.0505,  ...,  0.0586, -0.0982,  0.0789],
        ...,
        [-0.0814, -0.0721,  0.0699,  ...,  0.0469, -0.0058, -0.0259],
        [-0.0484,  0.0144,  0.0023,  ..., -0.0634,  0.0630, -0.0778],
        [-0.0473, -0.0545, -0.0820,  ..., -0.0897, -0.0772, -0.0822]])


In [95]:
# criterion = torch.nn.CrossEntropyLoss()
# criterion = F.nll_loss(log_softmax.view(1,-1), y_true)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [106]:
optimizer.zero_grad()
debug_iters = 2000
epochs = 2
for epoch in range(epochs):
    total_loss = 0.0
    for i in range(len(train_examples)):
        context_words, center_word = train_examples[i]
        
        input_ = torch.tensor([context_words])
        output_ = Variable(torch.from_numpy(np.array([center_word])).long())

        
        # Forward
        outputs = net(torch.tensor([input_]))
        log_softmax = F.log_softmax(outputs)
        
        # Backward
        loss = F.nll_loss(log_softmax,output_)
        loss.backward()
        
        # Optimize
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        if i % debug_iters == debug_iters-1:
            print(total_loss)
            total_loss = 0.0

/home/ankurshaswat/MyFiles/Repos/COL772/A1_part_1/env/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


12191.20909166336
15487.184662342072


KeyboardInterrupt: 

In [104]:
for name, param in net.named_parameters():
#     if param.requires_grad:
    print (name, param.data)

embed1.weight tensor([[-1.2616, -0.2915,  2.1652,  ...,  1.6965,  0.4838, -1.5713],
        [ 0.6393, -0.1153,  2.4186,  ...,  0.4038,  0.0099, -1.0737],
        [-0.7282, -0.6344, -0.1435,  ..., -1.7812, -0.4675, -0.1092],
        ...,
        [-0.2661, -0.1388,  1.4205,  ...,  0.1541,  0.3668, -0.1108],
        [-1.3831, -0.7770,  0.0425,  ..., -1.6575, -0.1327, -0.4992],
        [ 0.7098,  0.4363, -0.6173,  ..., -1.0384, -0.2999,  0.0213]])
linear1.weight tensor([[ 0.0302, -0.0173,  0.0682,  ..., -0.0449,  0.0550, -0.0740],
        [-0.0764,  0.0460, -0.0768,  ..., -0.0097, -0.0216,  0.0753],
        [-0.0557, -0.0987, -0.0504,  ...,  0.0584, -0.0979,  0.0785],
        ...,
        [-0.0820, -0.0722,  0.0692,  ...,  0.0465, -0.0058, -0.0256],
        [-0.0483,  0.0142,  0.0022,  ..., -0.0634,  0.0628, -0.0774],
        [-0.0474, -0.0542, -0.0818,  ..., -0.0898, -0.0771, -0.0814]])


In [105]:
print(len(train_examples))

4156346
